# Import

In [112]:
from PIL import Image
from os import listdir
from os.path import isfile, join
import numpy as np
import time


PATH = "D:\\NN\Images\\"
DIRS = ["Krajina", "Scan", "Noc", "Interier"]
# DIRS = ["Scan", "Noc", "Interier"]
NUMBER_OF_COLORS = 512
COLOR_RATIO = int(256/(NUMBER_OF_COLORS**(1/3)))

HISTOGRAMS = [[],[],[],[]]


# Načtení obrázků

In [116]:
HISTOGRAMS = [[],[],[],[]]

for i in range(len(DIRS)):
    files = [join(PATH + DIRS[i], f) for f in listdir(PATH + DIRS[i]) if isfile(join(PATH + DIRS[i], f))]
    for file in files:        
        image = Image.open(file)
        im = np.array(image)
        HISTOGRAMS[i].append(hist(im))
        
        
# print(HISTOGRAMS)

Hist :  47.79875588417053
Hist :  51.931843757629395
Hist :  0.005003213882446289


# Převod na 8-bit barevnou škálu

# Histogram

In [110]:
def hist(image):
    start_time = time.time()
    image = image//COLOR_RATIO
    
    base = 256//COLOR_RATIO
    base2 = base*base
    
    h = image.reshape(-1,3)
    h = h[:,0] + h[:,1]*base + h[:,2]*base2
    h = np.bincount(h)
    
    h = h/(image.shape[0]*image.shape[1])
    
    
    print("Hist2: ", (time.time() - start_time))
    return h    